<a href="https://colab.research.google.com/github/datta-base/grid-gallery/blob/master/The_dispark_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs


In [ ]:
#installs

!pip install mapclassify
!pip install geopandas
!pip install pyshp
!pip install descartes
!pip install fiona
!pip install numpy
!pip install dash --upgrade
!pip install folium --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.7 MB/s eta 0:00:00


# Imports, and file reading


In [ ]:
#imports

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
from osgeo import gdal
import json
from folium import GeoJson
from geopy.geocoders import Nominatim
import folium
from folium.features import CustomIcon
import imageio as iio

In [ ]:
#reading shpFile

ma = folium.Map(location = [40.735, -73.8235], zoom_start=2,
                tiles = 'https://api.mapbox.com/styles/v1/ctlee29/cks7vbgo627z718phehwpg5zk/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiY3RsZWUyOSIsImEiOiJja3JoM3UwNHo2YzJ1MzFtdHd0d2t6YXE4In0.JuM1D2_CbcgnrMl0v0PRIA',
                attr='Mapbox attribution')
disparkJson = '/content/Community Districts.geojson'
incomeData = pd.read_csv('/content/DisparkIncomeData.csv', engine = 'python')
shpFile = gpd.read_file('/content/nycdta2020.shp')
xls = pd.ExcelFile('/content/Dispark1.xlsx')
df1 = pd.read_excel('/content/Dispark1.xlsx')
fulldf = pd.read_excel('/content/FullData.xlsx')
dictxlsx = pd.ExcelFile('/content/DisparkDict.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/content/DisparkIncomeData.csv'

# Cleaning up data


In [ ]:
def remove_text_inside_brackets(text, brackets="()[]"):
    count = [0] * (len(brackets) // 2) # count open/close brackets
    saved_chars = []
    for character in text:
        for i, b in enumerate(brackets):
            if character == b: # found bracket
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close # `+1`: open, `-1`: close
                if count[kind] < 0: # unbalanced bracket
                    count[kind] = 0  # keep it
                else:  # found bracket to remove
                    break
        else: # character is not a [balanced] bracket
            if not any(count): # outside brackets
                saved_chars.append(character)
    return ''.join(saved_chars)

In [ ]:
corrected = []
index = 0

for i in shpFile['CDTAName']:
  corrected.append(repr(remove_text_inside_brackets(i[5:].lstrip()))[1:-2])
shpFile['CDTAName'] = corrected
print(shpFile['CDTAName'])

# Old methods



In [ ]:
df = dictxlsx.parse(xls.sheet_names[0])
dict1 = df.to_dict(orient = 'list')
print(dict1)

tempArr = []
for each in df1['Location.1']:
  if (hash(each) != 0):
    n = dict1[each][0]
    tempArr.append(n)
  else:
    break
df1['MedianIncome'] = tempArr

df1.to_csv('GraphData.csv')

In [ ]:
arrNames = df1['Location.1']
dictClean = dict.fromkeys(arrNames)
dictGreen = dict.fromkeys(arrNames)
dictMaint = dict.fromkeys(arrNames)
dictSaftey = dict.fromkeys(arrNames)
for index, row in fulldf.iterrows():
  if (hash(dictClean[row['Location.1']]) == 695334):
    dictClean.update({row['Location.1'] : row['Cleanliness']})
  else:
    dictClean.update({row['Location.1'] : (dictClean[row['Location.1']] + row['Cleanliness'])/2})
  if (hash(dictGreen[row['Location.1']]) == 695334):
    dictGreen.update({row['Location.1'] : row['Greenery']})
  else:
    dictGreen.update({row['Location.1'] : (dictGreen[row['Location.1']] + row['Greenery'])/2})
  if (hash(dictMaint[row['Location.1']]) == 695334):
    dictMaint.update({row['Location.1'] : row['Maintain']})
  else:
    dictMaint.update({row['Location.1'] : (dictMaint[row['Location.1']] + row['Maintain'])/2})
  if (hash(dictSaftey[row['Location.1']]) == 695334):
    dictSaftey.update({row['Location.1'] : row['Safety']})
  else:
    dictSaftey.update({row['Location.1'] : (dictSaftey[row['Location.1']] + row['Safety'])/2})
print(dictClean)
print(dictGreen)
print(dictMaint)
print(dictSaftey)

# Map Gen


In [ ]:
#ma.save('Map.html')
display(ma)

In [ ]:
ma = folium.Map([40.706253, -73.963896], zoom_start=11)
testArr = ['test1' , 'test2', 'test3']
folium.Choropleth(
    geo_data = disparkJson,
    data = incomeData,
    columns = ['Location', 'Data'],
    style_function=lambda feature: {
        'fillColor': '#ff0000',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5',
    },
    tooltips = 'feature.properties.boro_cd',
    popup = 'feature.properties.boro_cd',
    legend_name="Median Income",
).add_to(ma)

for index, row in incomeData.iterrows():
    lat = row["Coord1"]
    lon = row["Coord2"]
    if (row['Location'] in dictClean):
      folium.Marker(
        color='white',
        location=[lat, lon],
        tooltip= ("Name: " + str(row['Location'])
        + "<br>Median Income: $" + str(row['Data'])
        + "<br>Avg Safety Score: " + str(dictSaftey[row['Location']])
        + "<br>Avg Cleanliness Score: " + str(dictClean[row['Location']])
        + "<br>Avg Greenery Score: " + str(dictGreen[row['Location']])
        + "<br>Avg Maintenance Score: " + str(dictMaint[row['Location']]))
      ).add_to(ma)
    else:
      folium.Marker(
        color='white',
        location=[lat, lon],
        tooltip= ("Name: " + str(row['Location'])
        + "<br>Median Income: $" + str(row['Data'])
        + "<br>Avg Safety Score: N/A"
        + "<br>Avg Cleanliness Score: N/A"
        + "<br>Avg Greenery Score: N/A"
        + "<br>Avg Maintenance Score: N/A")
      ).add_to(ma)


folium.LayerControl().add_to(ma)